In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
plt.ion()

In [3]:
# Ouverture du fichier csv
name = ".data/SpotifyFeatures.csv"
data = pd.read_csv(name)

# Déselection des colonnes inutiles 
data = data.drop(columns = ['artist_name', 'track_name', 'track_id']) # les caractéristiques qui ne nous intéressent pas

# Compte le nombre de genres de musique (en enlevant "world" d'où le -1)
K=data.nunique()[0] - 1 

# Séparation des données entre les genres "World" et le reste
data_world = data.loc[data["genre"] == "World"]
data = data[data.genre != 'World']

# Nos données finales
y = data["genre"] 
X = data[data.columns.difference(["genre"])]

X
## structure de données:
# pkd: dimensions (K, D) matrice de la proba p(k,d) que le pixel d soit allumé pour les images de classe k
# Pk : dimension  (K)    matrice des probas de classe: P(y_true == k) est la proba qu'une image prise au hasard soit de classe k.

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,valence
0,0.61100,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0,0.0525,166.969,4/4,0.814
1,0.24600,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,1,0.0868,174.003,4/4,0.816
2,0.95200,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,3,0.0362,99.488,5/4,0.368
3,0.70300,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0,0.0395,171.758,4/4,0.227
4,0.95000,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,4,0.0456,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,0.00384,0.687,326240,0.714,0.544000,D,0.0845,-10.626,Major,39,0.0316,115.542,4/4,0.962
232721,0.03290,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,38,0.0337,113.830,4/4,0.969
232722,0.90100,0.517,166960,0.419,0.000000,D,0.0945,-8.282,Major,47,0.1480,84.135,4/4,0.813
232723,0.26200,0.745,222442,0.704,0.000000,A,0.3330,-7.137,Major,44,0.1460,100.031,4/4,0.489


In [15]:
## l'algo d'apprentissage ##
def BayesienNaif_fit(X,y):
    # y : yGT
    
    N = X.shape[0] # Le nombre de données
    D = X.shape[1] # Le nombre de pixels au total

    ## On crée nos matrices
    pkd = np.empty((K, D))
    Pk = np.zeros(K)
    
    # On les remplit
    # y == k # indices des images dont la classe est k
    #X[y==k] # image dont la classe est k
    
    for k in range (0, K):
        numerateur = np.sum(X[y==k], axis = 0) # objet de shape D
        denominateur = (y==k).sum() # décompte du nombre d'occurences de la classe k
        pkd[k,:] = numerateur[:] / denominateur # vecteur de taille D
        Pk[k] = denominateur
    ## devrait retourner les parametres du modele (appris)
    return pkd, Pk

pkd, Pk = BayesienNaif_fit(X,y)


## la fonction de dećision (a parametres fixés) ##
def BayesienNaif_predict(X,pkd, Pk):
    ypred = np.zeros(X.shape[0]) # ce que l'on prédit
    
    kEtoile = 0
    probaKmax = 0
    
    for n in range (0, X.shape[0]) : # Le nombre d'image
        probaKmax = 0
        for d in range (0, X.shape[1]) : # Le nombre de pixel dans une image
            for k in range (0, K) : 
                kEtoile = 1
                kEtoile =  X[n,d]*np.log(pkd[k, d] + 10**(-8))+(1-X[n,d])*np.log(1-pkd[k,d]+10**(-8))*np.log(Pk[k]/X.shape[0])
                if(probaKmax < kEtoile) :
                    ypred[n] = k
                    probaKmax = kEtoile
    return ypred # devrait retourner les labels des images

ypred = BayesienNaif_predict(X,pkd, Pk)
print(ypred)

KeyError: (0, 0)

In [ ]:
## separation train/test (ici, il n'y a pas d'hyperparametres, pratiquement, donc on ne fait pas de validation)
## quoique... il y en a au moins un, lequel ?
## TODO:
# Ntot   =      #Ntotal
# Ntrain =
# Ntest  =

X_train = X[0: Ntrain]
X_test  = X[Ntrain:Ntot] # X[-Ntest:] est equivalent
y_train = y[0:Ntrain]
y_test  = y[-Ntest:]

print("X.shape", X.shape)

In [ ]:
#############################
## petite demo d'affichage ##
plt.figure()
n=41 # image numero 42
plt.imshow(X_train[n].reshape(8,8) , cm.gray)
plt.title("ceci est censé ressembler à un "+str(y_train[n])+" apres la binarisation")
plt.show()
#############################

In [ ]:
## TODO: entrainer,

In [ ]:
## TODO: tester

In [ ]:
print("matrice de confusion: TODO - INDICE: lire la doc de sklearn.metrics / puis le faire a la main/plus tard(quand vous aurez sklearn)")


print("accuracy score: TODO -INDICE:  lire la doc de sklearn.metrics / puis le faire a la main (c'est facile)")


In [ ]:
###############################
## affichage des parametres: ##

## TODO: afficher les parametres appris sous un format lisible par l'humain

plt.show()

###############################